In [ ]:
#Optimization Example

In [2]:
import geopandas as gpd
import pandas as pd
import os, sys
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import LoadOSM as losm
import importlib
from pulp import LpInteger,LpVariable, LpProblem, lpSum, LpMinimize

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


In [3]:
osmNetwork = losm.OSM_to_network('./sampleData/nouakchott/mauritania-latest.osm.pbf')

In [4]:
osmNetwork.roads_raw.infra_type.value_counts()

residential       37459
track              5467
unclassified       4021
service            2028
path               1488
tertiary            891
footway             563
secondary           310
primary             228
construction        105
trunk                83
trunk_link           29
pedestrian           29
tertiary_link        14
primary_link          5
secondary_link        3
proposed              2
road                  2
raceway               2
cycleway              1
Name: infra_type, dtype: int64

In [5]:
accepted_road_types = ['residential', 'unclassified', 'track','service','tertiary','road','secondary','primary','trunk','primary_link','trunk_link','tertiary_link','secondary_link']

In [6]:
osmNetwork.filterRoads(acceptedRoads = accepted_road_types)

In [7]:
shp = gpd.read_file('./sampleData/nouakchott/layers/POLYGON.shp')
shp = shp.to_crs({'init':'epsg:4326'})
shp_obj = shp.geometry.iloc[0]

In [8]:
osmNetwork.roads_raw = osmNetwork.roads_raw.loc[osmNetwork.roads_raw.geometry.intersects(shp_obj) == True]

In [9]:
osmNetwork.generateRoadsGDF(verbose = False)

In [10]:
osmNetwork.initialReadIn()

In [11]:
gn.save(osmNetwork.network,'nouakchott','./sampleData/nouakchott/')

In [12]:
#the cleaning network part
import os, sys, time
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

In [13]:
def CleanNetwork(G, wpath, country, UTM, WGS = {'init': 'epsg:4326'}, junctdist = 50, verbose = False):
    
    ### Topologically simplifies an input graph object by collapsing junctions and removing interstital nodes
    # REQUIRED - G: a graph object containing nodes and edges. edges should have a property 
    #               called 'Wkt' containing geometry objects describing the roads
    #            wpath: the write path - a drive directory for inputs and output
    #            country: this parameter allows for the sequential processing of multiple countries
    #            UTM: the epsg code of the projection, in metres, to apply the junctdist
    # OPTIONAL - junctdist: distance within which to collapse neighboring nodes. simplifies junctions. 
    #            Set to 0.1 if not simplification desired. 50m good for national (primary / secondary) networks
    #            verbose: if True, saves down intermediate stages for dissection
    ################################################################################################
    
    # Squeezes clusters of nodes down to a single node if they are within the snapping tolerance
    a = gn.simplify_junctions(G, UTM, WGS, junctdist)

    # ensures all streets are two-way
    a = gn.add_missing_reflected_edges(a)
    
    #save progress
    if verbose is True: 
        gn.save(a, 'a', wpath)
    
    # Finds and deletes interstital nodes based on node degree
    b = gn.custom_simplify(a)
    
    # rectify geometry
    for u, v, data in b.edges(data = True):
        if type(data['Wkt']) == list:
                data['Wkt'] = gn.unbundle_geometry(data['Wkt'])
    
    # save progress
    if verbose is True: 
        gn.save(b, 'b', wpath)
    
    # For some reason CustomSimplify doesn't return a MultiDiGraph. Fix that here
    c = gn.convert_to_MultiDiGraph(b)

    # This is the most controversial function - removes duplicated edges. This takes care of two-lane but separate highways, BUT
    # destroys internal loops within roads. Can be run with or without this line
    c = gn.remove_duplicate_edges(c)

    # Run this again after removing duplicated edges
    c = gn.custom_simplify(c)

    # Ensure all remaining edges are duplicated (two-way streets)
    c = gn.add_missing_reflected_edges(c)
    
    # save final
    gn.save(c, '%s_processed' % country, wpath)
    
    print('Edge reduction: %s to %s (%d percent)' % (G.number_of_edges(), 
                                               c.number_of_edges(), 
                                               ((G.number_of_edges() - c.number_of_edges())/G.number_of_edges()*100)))
    return c

In [14]:
UTMZs = {'MRT':32628}

WGS = {'init': 'epsg:4326'}

countries = ['MRT']

wpath = r'./sampleData/nouakchott/'

for country in countries:
    
    print('\n--- processing for: %s ---\n' % country)
    print('start: %s\n' % time.ctime())

    print('Outputs can be found at: %s\n' % (wpath))
        
    UTM = {'init': 'epsg:%d' % UTMZs[country]}
    
    G = nx.read_gpickle(os.path.join(wpath, 'nouakchott.pickle'))
    
    G = CleanNetwork(G, wpath, country, UTM, WGS, 0.5, verbose = False)
    print('\nend: %s' % time.ctime())
    print('\n--- processing complete for: %s ---' % country)


--- processing for: MRT ---

start: Fri Mar 22 16:27:26 2019

Outputs can be found at: ./sampleData/nouakchott/

60139
120238
118392
59249
118388
Edge reduction: 60142 to 118388 (-96 percent)

end: Fri Mar 22 16:33:28 2019

--- processing complete for: MRT ---


In [15]:
#net prep phase

In [16]:
#read back in processed graph
G = nx.read_gpickle('./sampleData/nouakchott/MRT_processed.pickle')

In [17]:
G.number_of_edges()

118388

In [18]:
G.number_of_nodes()

37305

In [19]:
sd = {          'residential': 10,  # kmph
                'primary': 25, # kmph
                'primary_link':20,
                'motorway':35,
                'motorway_link': 25,
                'trunk': 20,
                'trunk_link':15,
                'secondary': 10, # kmph
                'secondary_link':5,
                'tertiary':5,
                'tertiary_link': 5,
                'unclassified':5
                }

In [20]:
gn.example_edge(G)

(0, 24505, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 0.03882328422750167, 'Type': 'legitimate'})


In [21]:
#look into modifying this for weighted distances
G_time = gn.convert_network_to_time(G, 
                                   distance_tag = 'length',
                                   graph_type = 'drive', 
                                   road_col = 'infra_type',
                                   speed_dict = sd, 
                                   factor = 1000)

In [22]:
gn.example_edge(G_time)

(0, 24505, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 38.82328422750167, 'Type': 'legitimate', 'time': 13.976382321900601, 'mode': 'drive'})


In [23]:
D = list(nx.strongly_connected_component_subgraphs(G_time))

In [24]:
len(D)

11

In [25]:
G = D[0]

In [26]:
G = nx.convert_node_labels_to_integers(G)

In [27]:
gn.save(G, 'biggest_subg', './sampleData/nouakchott/')

In [3]:
#make sure origin and destination are created. They will be CSVs with 'Lat' and 'Lon' columns.
#There will also need to be a way in the future to add a 'demand' column for the origins CSV, and a 'capacity' column for the destinations dataset.
origins = pd.read_csv('./sampleData/nouakchott/origins_test1.csv')


NameError: name 'pd' is not defined

In [29]:
origins['geometry'] = list(zip(origins['Lon'],origins['Lat']))
origins['geometry'] = origins['geometry'].apply(Point)
origins_gdf = gpd.GeoDataFrame(origins, crs = {'init':'epsg:4326'}, geometry = 'geometry')
origins_gdf = gn.pandana_snap(G, origins_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
origins = list(origins_gdf.NN)
origins = list(set(origins))

/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1580: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  G_tree = spatial.KDTree(node_gdf[['x','y']].as_matrix())
/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1582: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indices = G_tree.query(in_df[['x','y']].as_matrix())


In [124]:
origins_gdf

,Lon,Lat,id,demand,geometry,NN,NN_dist
0,-15.920091,18.068748,0,492971,POINT (-15.9200905444876 18.068747698414),5343,38.598697
1,-15.970769,18.011527,1,455613,POINT (-15.9707686005727 18.0115274721116),15307,40.916068
2,-16.017149,18.101601,2,461385,POINT (-16.0171494105211 18.1016011324086),32797,13.793765
3,-16.011744,18.080487,3,253764,POINT (-16.0117439165805 18.0804867868114),19488,93.794886
4,-15.874072,18.065438,4,182095,POINT (-15.8740715454476 18.065437965025),11633,199.641930
5,-15.936539,18.142234,5,325801,POINT (-15.9365393885209 18.1422342764535),34556,33.226094
6,-15.936032,18.035973,6,216626,POINT (-15.9360316762384 18.0359726230182),34965,28.303928
7,-15.989495,18.010282,7,175802,POINT (-15.989494705491 18.0102817599499),5702,18.700289
8,-15.988440,18.062400,8,161223,POINT (-15.9884396394221 18.062399556345),11029,12.908252
9,-15.979721,17.992657,9,196191,POINT (-15.979721387056 17.9926573776181),12121,45.940805


In [30]:
destinations = pd.read_csv('./sampleData/nouakchott/destinations_test1.csv')

In [31]:
destinations['geometry'] = list(zip(destinations['Lon'],destinations['Lat']))
destinations['geometry'] = destinations['geometry'].apply(Point)
destinations_gdf = gpd.GeoDataFrame(destinations, crs = {'init':'epsg:4326'}, geometry = 'geometry')
destinations_gdf = gn.pandana_snap(G, destinations_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
#destinations_gdf.NN is the nearest node of the road network
destinations = list(destinations_gdf.NN)
destinations = list(set(destinations))

In [32]:
destinations_gdf

,Lon,Lat,id,Unnamed: 3,geometry,NN,NN_dist
0,-16.005524,18.133726,1,NaN,POINT (-16.0055244876789 18.1337264869769),25486,244.940598
1,-15.911223,18.146442,2,NaN,POINT (-15.9112234904605 18.1464419959018),6724,91.625141
2,-15.894339,18.046510,3,NaN,POINT (-15.8943385146072 18.0465095569873),1646,26.534798
3,-15.966657,17.979553,4,NaN,POINT (-15.9666571847713 17.979553320176),8994,21.888530
4,-15.980675,18.028031,5,NaN,POINT (-15.980674900574 18.0280309665427),15739,34.104235
5,-16.002020,18.067712,6,NaN,POINT (-16.0020200587282 18.0677121043953),33112,31.657860
6,-16.002020,18.104962,7,NaN,POINT (-16.0020200587282 18.1049618080371),4092,52.440472
7,-15.977170,18.084672,8,NaN,POINT (-15.9771704716234 18.0846722997527),4965,54.837685
8,-15.941489,18.061654,9,NaN,POINT (-15.9414890132164 18.0616544947466),2440,5.547694
9,-15.954232,18.110110,10,NaN,POINT (-15.9542323912189 18.1101095187286),7011,6.335953


In [33]:
print("length of origins is %s" % len(origins))

length of origins is 50


In [34]:
print("length of destinations is %s" % len(destinations))

length of destinations is 10


In [35]:
len(origins) * len(destinations)

500

In [36]:
d_test = destinations[:50]
print(d_test)

[8994, 7011, 6724, 4965, 2440, 1646, 25486, 33112, 15739, 4092]


In [37]:
#look more into this calculate_OD function
%time OD = gn.calculate_OD(G, origins, destinations, fail_value = 9999999999999)

CPU times: user 2.34 s, sys: 5.34 ms, total: 2.34 s
Wall time: 2.51 s


In [38]:
origins_df = pd.DataFrame(OD, columns = destinations, index = origins)

In [39]:
facilities = origins_df.columns.values.tolist()

In [40]:
facilities

[8994, 7011, 6724, 4965, 2440, 1646, 25486, 33112, 15739, 4092]

In [41]:
result = gn.optimize_facility_locations(origins_df, facilities, 4, existing_facilities = None)

In [42]:
result

[15739, 2440, 4092, 7011]

In [56]:
#result is 5,9,7,10

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


<module 'GOSTnet' from '/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py'>

In [2]:
import importlib
importlib.reload(gn)

NameError: name 'gn' is not defined

In [1]:
result2 = gn.optimize_set_coverage(origins_df)

NameError: name 'gn' is not defined

In [120]:
result2

[1646]

In [121]:
#Now will re-run analysis using weighted demands
d_test = destinations[:50]
print(d_test)

[8994, 7011, 6724, 4965, 2440, 1646, 25486, 33112, 15739, 4092]


In [125]:
origins_gdf

,Lon,Lat,id,demand,geometry,NN,NN_dist
0,-15.920091,18.068748,0,492971,POINT (-15.9200905444876 18.068747698414),5343,38.598697
1,-15.970769,18.011527,1,455613,POINT (-15.9707686005727 18.0115274721116),15307,40.916068
2,-16.017149,18.101601,2,461385,POINT (-16.0171494105211 18.1016011324086),32797,13.793765
3,-16.011744,18.080487,3,253764,POINT (-16.0117439165805 18.0804867868114),19488,93.794886
4,-15.874072,18.065438,4,182095,POINT (-15.8740715454476 18.065437965025),11633,199.641930
5,-15.936539,18.142234,5,325801,POINT (-15.9365393885209 18.1422342764535),34556,33.226094
6,-15.936032,18.035973,6,216626,POINT (-15.9360316762384 18.0359726230182),34965,28.303928
7,-15.989495,18.010282,7,175802,POINT (-15.989494705491 18.0102817599499),5702,18.700289
8,-15.988440,18.062400,8,161223,POINT (-15.9884396394221 18.062399556345),11029,12.908252
9,-15.979721,17.992657,9,196191,POINT (-15.979721387056 17.9926573776181),12121,45.940805


In [137]:
origins_w_demands = pd.Series(origins_gdf.demand.values,index=origins_gdf.NN).to_dict()
#origins
#origins = list(set(origins))

In [138]:
origins_w_demands

{5343: 492971,
 15307: 455613,
 32797: 461385,
 19488: 253764,
 11633: 182095,
 34556: 325801,
 34965: 216626,
 5702: 175802,
 11029: 161223,
 12121: 196191,
 33435: 310357,
 22552: 122339,
 7178: 267360,
 20596: 96373,
 5048: 86563,
 16726: 187160,
 35016: 55439,
 32092: 361740,
 27750: 208580,
 4798: 353201,
 9699: 58399,
 33926: 463887,
 27222: 267470,
 10098: 482864,
 5307: 341719,
 27040: 79911,
 4851: 63988,
 18033: 465259,
 14482: 342292,
 15243: 67766,
 11322: 466562,
 8638: 322844,
 2044: 404398,
 23723: 310314,
 28291: 333495,
 15877: 435117,
 32820: 320200,
 17189: 285849,
 26830: 364402,
 26977: 118238,
 31973: 329939,
 9703: 90160,
 30616: 213393,
 18544: 168285,
 31222: 344975,
 32893: 409069,
 4101: 422713,
 11310: 342932,
 27867: 227197,
 36602: 285536}

In [162]:
import importlib
importlib.reload(gn)

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


<module 'GOSTnet' from '/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py'>

In [163]:
#look more into this calculate_OD function
%time OD_weighted_demand = gn.calculate_OD(G, origins_w_demands, destinations, fail_value = 9999999999999, weighted_origins=True)

print origins type
<class 'dict'>
weighted_origins equals true
CPU times: user 1min 2s, sys: 0 ns, total: 1min 2s
Wall time: 1min 3s


In [164]:
len(OD_weighted_demand)

50

In [165]:
OD_weighted_demand_df = pd.DataFrame(OD_weighted_demand, columns = destinations, index = origins)

In [166]:
OD_weighted_demand_df

,8994,7011,6724,4965,2440,1646,25486,33112,15739,4092
28291,1.511662e+09,8.236943e+08,1.086134e+09,6.734612e+08,3.885154e+08,6.798848e+08,1.247767e+09,1.215590e+09,1.043079e+09,1.013181e+09
15877,5.468000e+08,1.182195e+09,1.424747e+09,1.043346e+09,1.143932e+09,1.686910e+09,1.574131e+09,1.131587e+09,4.093913e+08,1.357322e+09
33926,1.939196e+09,1.130897e+09,1.376522e+09,7.326614e+08,1.266827e+09,1.806418e+09,6.740431e+08,8.024236e+08,1.498893e+09,3.671309e+08
4101,9.514361e+08,5.829689e+08,7.180637e+08,3.639372e+08,6.577310e+08,9.545084e+08,4.485929e+08,1.659039e+08,7.043929e+08,3.278361e+08
7178,7.171686e+08,4.627383e+08,5.596792e+08,4.072448e+08,3.022977e+08,2.416623e+08,6.193836e+08,6.078048e+08,5.440824e+08,5.327314e+08
15243,1.308367e+09,4.965221e+08,3.026478e+08,6.386428e+08,8.335825e+08,1.214607e+09,1.017864e+09,1.028307e+09,9.979517e+08,8.628278e+08
14482,6.455522e+08,6.531022e+08,7.684261e+08,5.870854e+08,2.954499e+08,4.773421e+08,8.394524e+08,6.831663e+08,4.408060e+08,7.363683e+08
34965,2.810809e+08,5.160977e+08,6.096884e+08,4.622526e+08,5.013575e+08,7.108702e+08,6.670602e+08,4.957529e+08,1.674057e+08,5.834026e+08
11029,4.112446e+08,3.367964e+08,4.226257e+08,2.006149e+08,3.271865e+08,5.157572e+08,3.729736e+08,1.181856e+08,2.542915e+08,2.962536e+08
22552,1.372281e+08,5.655748e+08,6.700198e+08,5.056813e+08,5.490985e+08,7.829098e+08,7.342418e+08,5.430668e+08,2.320828e+08,6.408819e+08


In [ ]:
facilities = OD_weighted_demand_df.columns.values.tolist()

In [167]:
result_w_demands = gn.optimize_facility_locations(OD_weighted_demand_df, facilities, 4, existing_facilities = None)

In [168]:
result_w_demands

[15739, 2440, 4092, 7011]